In [1]:
# import prereq.
from __future__ import absolute_import, division, print_function, unicode_literals
 
import numpy as np
import pandas as pd 
import tensorflow as tf

from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler # used for callback 


In [2]:

X = pd.read_csv('X.csv')
Y = pd.read_csv('Y.csv')

X_c = pd.read_csv('X_c.csv')
Y_c = pd.read_csv('Y_c.csv')

X_t = pd.read_csv('X_t.csv')
Y_t = pd.read_csv('Y_t.csv')

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
# convert to numpy  np.append(a, [[7,8,9]],axis = 0)
data_type =  np.dtype(np.float64) # define datatype

x = np.asarray(X.iloc[:,1:].to_numpy(), dtype=data_type)
y = np.asarray(Y.iloc[:,1].to_numpy(), dtype=data_type)
x_t = np.asarray(X_t.iloc[:,1:].to_numpy(), dtype=data_type)
y_t = np.asarray(Y_t.iloc[:,1].to_numpy(), dtype=data_type)
x_c = np.asarray(X_c.iloc[:,1:].to_numpy(), dtype=data_type)
y_c = np.asarray(Y_c.iloc[:,1].to_numpy(), dtype=data_type)


# clean some memory
X = X_c = X_t = Y = Y_c = Y_t = None

In [5]:
x_train = x.reshape(x.shape[0], img_rows, img_cols, 1)
x_cross = x_c.reshape(x_c.shape[0], img_rows, img_cols, 1)
x_test = x_t.reshape(x_t.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols,1)

In [6]:
x_train = x_train.astype('float32')
x_cross = x_cross.astype('float32')
x_test = x_test.astype('float32')


In [7]:
y_train = keras.utils.to_categorical(y, num_classes)
y_cross = keras.utils.to_categorical(y_c, num_classes)
y_test = keras.utils.to_categorical(y_t, num_classes)

In [8]:
y_test.shape

(350, 10)

In [9]:
# arhitect the model
# CGG - LITE 28x28 MAS
model = tf.keras.Sequential([

# layers.Conv2D(8, kernel_size=(7, 7),
#                  padding='same',
#                  activation='relu',
#                  input_shape=input_shape),  
# layers.Dropout(0.7),
# # layers.Dropout(0.5),
# # layers.MaxPooling2D(pool_size=(2, 2)),
# # layers.Conv2D(16, padding='same', kernel_size=(14, 14), activation='relu'),
# # layers.Dropout(0.7),
# # layers.MaxPooling2D(pool_size=(2, 2)),
    
# # layers.MaxPooling2D(pool_size=(2, 2)),
# # layers.Conv2D(16, padding='same',kernel_size=(1, 3), activation='relu'),
# # layers.Dropout(0.45),
    
# layers.Flatten(),

# # layers.Dense(1024, activation='relu'),#1024
# # layers.Dropout(0.5),
    
# layers.Dense(3072, activation='relu'),#1024
# layers.Dropout(0.6),
    
# layers.Dense(3072, activation='relu'),#1024
# layers.Dropout(0.6),
    

# layers.Dense(400, activation='relu'),#1024
# layers.Dropout(0.55),

# layers.Dense(100, activation='relu'),#1024
# layers.Dropout(0.5),
    
# layers.Dense(10, activation='softmax')]) -- best
    
    
layers.Conv2D(10, kernel_size=(7, 7),
                 padding='same',
                 activation='relu',
                 input_shape=input_shape),  
layers.Dropout(0.65),
# layers.Dropout(0.5),
# layers.MaxPooling2D(pool_size=(2, 2)),
# layers.MaxPooling2D(pool_size=(2, 2)),
    
# layers.MaxPooling2D(pool_size=(2, 2)),
# layers.Conv2D(16, padding='same',kernel_size=(1, 3), activation='relu'),
# layers.Dropout(0.45),
    
layers.Flatten(),

# layers.Dense(1024, activation='relu'),#1024
# layers.Dropout(0.5),
    
layers.Dense(3072, activation='relu'),#1024
layers.Dropout(0.62),
    
layers.Dense(3072, activation='relu'),#1024
layers.Dropout(0.62),
    
layers.Dense(3072, activation='relu'),#1024
layers.Dropout(0.62),
      
layers.Dense(400, activation='relu'),#1024
layers.Dropout(0.52),
    
layers.Dense(100, activation='relu'),#1024
layers.Dropout(0.52),
        
layers.Dense(10, activation='softmax')])
# max 90  acc

# 69300/69300 [==============================] - 13s 181us/sample - loss: 0.0626 - accuracy: 0.9767 - val_loss: 0.4450 - val_accuracy: 0.9143
# Out[48]:
# <tensorflow.python.keras.callbacks.History at 0x1349aba3a48>

In [10]:
def step_decay(epoch):
    initial_lrate = 0.001
    k = 1
    return k / np.sqrt(epoch+1) * initial_lrate
     

lrate = LearningRateScheduler(step_decay)

In [11]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
 
callbacks = [EarlyStopping(monitor='val_accuracy',verbose=1,min_delta=0.00000001, patience=90), lrate]

In [12]:
model.fit(x_train,y_train, 
          epochs=100,
          batch_size=100,
          shuffle=True, 
          verbose=1,
         validation_data=(x_cross, y_cross), callbacks=callbacks)

Train on 277200 samples, validate on 1400 samples
Epoch 1/100
277200/277200 [==============================] - 128s 462us/sample - loss: 0.7981 - accuracy: 0.7126 - val_loss: 0.3391 - val_accuracy: 0.8786
Epoch 2/100
277200/277200 [==============================] - 126s 456us/sample - loss: 0.4893 - accuracy: 0.8373 - val_loss: 0.3367 - val_accuracy: 0.8943
Epoch 3/100
277200/277200 [==============================] - 126s 455us/sample - loss: 0.4261 - accuracy: 0.8579 - val_loss: 0.2657 - val_accuracy: 0.9100
Epoch 4/100
277200/277200 [==============================] - 126s 455us/sample - loss: 0.3880 - accuracy: 0.8694 - val_loss: 0.2599 - val_accuracy: 0.9043
Epoch 5/100
277200/277200 [==============================] - 126s 455us/sample - loss: 0.3617 - accuracy: 0.8773 - val_loss: 0.2387 - val_accuracy: 0.9086
Epoch 6/100
277200/277200 [==============================] - 126s 455us/sample - loss: 0.3410 - accuracy: 0.8841 - val_loss: 0.2427 - val_accuracy: 0.9086
Epoch 7/100
277200/2

In [15]:
model.evaluate(x_test, y_test)

350/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2505875767128808, 0.93142855]

In [16]:
model.save('97_935_93.h5')